<a href="https://colab.research.google.com/github/musicjae/Reinforcement_Learning/blob/master/policy_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 컴퓨터 내에서 특정 파일 마운트 하기
from google.colab import files
uploaded = files.upload()

In [ ]:
# 구글 드라이브 전체와 마운트 하기
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
!ls

 **Intro**<br><br>
 &nbsp; &nbsp;보통 The Bellman equation은 dynamic programming (policy_iteration, value_iteration)을 통해 풀 수 있다. 이번 실행에서 우리는 Bellman eq를 사용해 policy iteration을 풀어볼 것이다.
<br><br>
 **Assumption**<br><br>

 &nbsp; &nbsp; (1) The optimal policy is not random. We need to develop the present policy which is called Policy Improvement. <br>
 &nbsp; &nbsp;(2) By implemetation of policy evaluation, we should develop the present Improvement. Thus, we need to consider what is the policy evaluation.<br><br>
 **Policy Evaluation**

&nbsp; &nbsp;PE? 특정 정책이 얼마나 좋은지에 대해 평가하기. <br>
&nbsp; &nbsp;Policy value? 현재 정책에 따라 받을 보상에 대한 정보.<Br><Br><Br>
$$(1)\space\space v_{\pi}(s) = E_{\pi}[R_{t+1}+ \gamma R_{t+2}+ (\gamma)^2R_{t+3}+ ... | S_t = s]$$
<br><br>
 하지만 컴퓨터는 위 식을 계산하지 못한다. 우리는 이것을 계산 가능한 식으로 바꿔줘야 한다. 그 식은 다음과 같다:<br><br>
 $$(2)\space\space  v_{\pi}(s) = \sum_{a∈A}\pi(a|s)(r_{(s,a)} + \gamma v_{\pi}(S'))$$
<br><br>
정책 평가 PE 는 위 식을 반복적으로 수행하는 것이다. 우리가 $\pi$를  새로운 변수 $k=1,2, ...$로 표현할 때, $k+1$ 번째 가치함수에 대한 방정식은 다음과 같다:<br><br>
$$(3)\space\space v_{k+1}(s) = \sum_{a∈A}\pi(a|s)(r_{(s,a)} + \gamma v_{k}(S'))$$

**Details of (3):**<br><br>
- k th의 가치함수 행렬에서 현재 상태 s에서 갈 수 있는 다음 상태 s' 에 저장된 가치 함수 $v_k(s')$ 불러오기
-  그 가치 함수에 할인율을 곱한 뒤, 그 상태로 가는 행동에 대한 보상 R을 더한다:<Br>
$$(4)\space\space r_{(s,a)} + \gamma v_k(s')$$<Br>
- (4)에다가 그 행동을 할 확률 (정책) 을 곱한다:<br><Br>
$$(5)\space\space \pi(a|s)(r_{(s,a)} + \gamma v_k(s'))$$<Br>
- (5)를 ALL 선택 가능한 행동에 대해 반복하고 그 값들을 더해준다:<br><br>
$$(6)\space\space \sum_{a∈A}\pi(a|s)r_{(s,a)} + \gamma v_k(s')$$<Br>
- (6) 과정을 통해 얻은 값을 k+1 th 가치 함수 행렬의 상태 s 자리에 저장해준다.


In [ ]:

import numpy as np
from environment import GraphicDisplay, Env


class PolicyIteration:
    def __init__(self, env):
        # 환경에 대한 객체 선언
        self.env = env
        # 가치함수를 2차원 리스트로 초기화
        self.value_table = [[0.0] * env.width for _ in range(env.height)]
        # 상 하 좌 우 동일한 확률로 정책 초기화
        self.policy_table = [[[0.25, 0.25, 0.25, 0.25]] * env.width
                             for _ in range(env.height)]
        # 마침 상태의 설정
        self.policy_table[2][2] = []
        # 할인율
        self.discount_factor = 0.9

    # 벨만 기대 방정식을 통해 다음 가치함수를 계산하는 정책 평가
    def policy_evaluation(self):
        # 다음 가치함수 초기화
        next_value_table = [[0.00] * self.env.width
                            for _ in range(self.env.height)]

        # 모든 상태에 대해서 벨만 기대방정식을 계산
        for state in self.env.get_all_states(): # 에이전트는 가능한 한 모든 상태에 대해 알아야 하니까
            value = 0.0
            # 마침 상태의 가치 함수 = 0
            if state == [2, 2]:
                next_value_table[state[0]][state[1]] = value
                continue

            # 벨만 기대 방정식
            for action in self.env.possible_actions: # 에이전트의 모든 행동
                next_state = self.env.state_after_action(state, action) # 에이전트가 특정 상태에서 특정 행동을 할 때, 다음 상태에 대한 환경 정보
                reward = self.env.get_reward(state, action) # 환경이 주는 보상
                next_value = self.get_value(next_state)
                value += (self.get_policy(state)[action] * 
                          (reward + self.discount_factor * next_value))

            next_value_table[state[0]][state[1]] = value

        self.value_table = next_value_table

    # 현재 가치 함수에 대해서 탐욕 정책 발전
    def policy_improvement(self):
        next_policy = self.policy_table
        for state in self.env.get_all_states():
            if state == [2, 2]:
                continue

            value_list = []
            # 반환할 정책 초기화
            result = [0.0, 0.0, 0.0, 0.0]

            # 모든 행동에 대해서 [보상 + (할인율 * 다음 상태 가치함수)] 계산
            for index, action in enumerate(self.env.possible_actions):
                next_state = self.env.state_after_action(state, action)
                reward = self.env.get_reward(state, action)
                next_value = self.get_value(next_state)
                value = reward + self.discount_factor * next_value
                value_list.append(value)

            # 받을 보상이 최대인 행동들에 대해 탐욕 정책 발전
            max_idx_list = np.argwhere(value_list == np.amax(value_list))
            max_idx_list = max_idx_list.flatten().tolist()
            prob = 1 / len(max_idx_list)

            for idx in max_idx_list:
                result[idx] = prob

            next_policy[state[0]][state[1]] = result

        self.policy_table = next_policy

    # 특정 상태에서 정책에 따라 무작위로 행동을 반환
    def get_action(self, state):
        policy = self.get_policy(state)
        policy = np.array(policy)
        return np.random.choice(4, 1, p=policy)[0]

    # 상태에 따른 정책 반환
    def get_policy(self, state):
        return self.policy_table[state[0]][state[1]]

    # 가치 함수의 값을 반환
    def get_value(self, state):
        return self.value_table[state[0]][state[1]]


if __name__ == "__main__":
    env = Env() # 에이전트에게 환경에 대한 정보가 필요하니까 env를 객체로 생성
    policy_iteration = PolicyIteration(env) #이 env를 정책 반복 클래스의 인수로 전달 -> 환경의 env() 클래스에 접근 가능해져
    grid_world = GraphicDisplay(policy_iteration)
    grid_world.mainloop()